In [25]:
import pandas as pd
import os
import dotenv as dotenv
from time import sleep
import sys
import os
# Asegúrate de que ../src esté en el path
sys.path.append(os.path.join(os.path.dirname(os.path.abspath('')), '../src'))
# Intenta importar la función
from dic import diccionario_correspondencia1
from dic import diccionario_correspondencia2
from dic import diccionario_correspondencia3

In [26]:
df_clasidicacion=pd.read_csv('../../data/base/clasificaciones_premier.csv')
df_info=pd.read_csv('../../data/base/info_clubes.csv')
df_estadios=pd.read_csv('../../data/base/asistencia_estadios.csv')
df_jugadores=pd.read_csv('../../data/base/players_season_[2005].csv')

In [27]:
len(sorted(df_jugadores['team'].unique()))

26

In [28]:
len(sorted(df_clasidicacion['equipo'].unique()))

43

In [29]:
len(sorted(df_info['Nombre Completo'].unique()))

23

In [30]:
len(sorted(df_estadios['Equipo'].unique()))

51

Crear lista de equipos

In [31]:
lista_equipos=sorted(df_estadios['Equipo'].unique())

In [32]:
df_clubes=pd.DataFrame(lista_equipos,columns=['Equipos'])

Homogeneizar para que todos los df tengan la misma columna de Equipo definida

In [33]:
df_clasidicacion['equipo'] = df_clasidicacion['equipo'].map(diccionario_correspondencia1)

In [34]:
df_info['Equipo']=df_info['Nombre Completo'].map(diccionario_correspondencia2)

In [35]:
df_jugadores['team']=df_jugadores['team'].map(diccionario_correspondencia3)

Traducir las columnas del df_jugadores

In [36]:
# Diccionario de traducción de columnas
columnas_traducidas = {
    'season': 'temporada',
    'player_id': 'id_jugador',
    'player_name': 'nombre_jugador',
    'age': 'edad',
    'nationality': 'nacionalidad',
    'team': 'equipo',
    'games_played': 'partidos_jugados',
    'minutes': 'minutos',
    'goals': 'goles',
    'yellow_cards': 'tarjetas_amarillas',
    'red_cards': 'tarjetas_rojas'
}

# Aplicar las traducciones a las columnas de un DataFrame (df)
df_jugadores = df_jugadores.rename(columns=columnas_traducidas)


Homogeneizar para que todos los df tengan la misma columna de Temporada

In [37]:
df_clasidicacion['temporada'] = df_clasidicacion['temporada'].str.split('-').str[0].astype(int)

In [38]:
# Función para convertir el formato "92/93" a "1992"
def convertir_temporada(temporada):
    # Tomar los primeros dos dígitos del año inicial
    anio_inicio = int(temporada.split('/')[0])
    # Si el año es menor a 50, asumir que es 2000s; si es mayor, es 1900s
    if anio_inicio < 50:
        return 2000 + anio_inicio
    else:
        return 1900 + anio_inicio

In [39]:
df_estadios['Temporada'] = df_estadios['Temporada'].apply(convertir_temporada)

Homogeneizar datos de rendimiento en Estadios

In [40]:
# Limpiar la columna y convertir a float
df_estadios['Máximo rendimiento'] = df_estadios['Máximo rendimiento'].str.replace(',', '.').str.replace('%', '').astype(float)

# Renombrar la columna
df_estadios.rename(columns={'Máximo rendimiento	': 'Máximo rendimiento (%)'}, inplace=True)

Rellenar NaN de lleno en df_estadios

In [41]:
df_estadios.fillna(0, inplace=True)

Rellenar NaN de titulos en df_info

In [42]:
df_info.fillna(0, inplace=True)

In [43]:
df_info[['Recopa de Europa', "Full Members' Cup"]] = df_info[['Recopa de Europa', "Full Members' Cup"]].astype(int)


Borrar columna de asistencias en df_jugadores

In [44]:
df_jugadores=df_jugadores.drop(columns='assists')

In [45]:
df_jugadores.to_csv(f'../../data/transformed/stats_jugadores_transformed.csv', index=False)
df_info.to_csv(f'../../data/transformed/info_clubes_transformed.csv', index=False)
df_clasidicacion.to_csv(f'../../data/transformed/clasificaciones_premier_transformed.csv', index=False)
df_estadios.to_csv(f'../../data/transformed/asistencia_estadios_transformed.csv', index=False)
df_clubes.to_csv(f'../../data/transformed/clubes.csv', index=False)